In [ ]:
## Libraries
import os
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from tkinterdnd2 import TkinterDnD, DND_FILES
# from tkinter.dnd import TkinterDnD
from PyPDF2 import PdfMerger


In [2]:


class PDFMergerApp(TkinterDnD.Tk):
    def __init__(self):
        super().__init__()
        self.title("Combinador de PDFs")
        self.geometry("600x500")
        self.configure(bg='#f0f0f0')
        
        # Variables
        self.pdf_files = []
        
        # Estilo
        self.style = ttk.Style()
        self.style.configure('TButton', padding=5, font=('Arial', 10))
        self.style.configure('TLabel', padding=5, font=('Arial', 10))
        
        # Widgets
        self.create_widgets()
        
    def create_widgets(self):
        # Frame principal
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Label de instrucciones
        lbl_instructions = ttk.Label(main_frame, text="Arrastra y suelta archivos PDF aquí o usa el botón 'Agregar'")
        lbl_instructions.pack(pady=(0, 10))
        
        # Listbox para mostrar los PDFs
        self.listbox = tk.Listbox(
            main_frame, 
            selectmode=tk.SINGLE, 
            height=10,
            width=60,
            bg='white',
            font=('Arial', 10),
            relief=tk.SUNKEN,
            highlightthickness=0
        )
        self.listbox.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        # Configurar DnD
        self.listbox.drop_target_register('DND_Files')
        self.listbox.dnd_bind('<<Drop>>', self.drop)
        
        # Frame de botones de control
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Botones de control
        btn_add = ttk.Button(control_frame, text="Agregar PDF(s)", command=self.add_files)
        btn_add.pack(side=tk.LEFT, padx=5)
        
        btn_remove = ttk.Button(control_frame, text="Eliminar", command=self.remove_file)
        btn_remove.pack(side=tk.LEFT, padx=5)
        
        btn_up = ttk.Button(control_frame, text="Subir", command=self.move_up)
        btn_up.pack(side=tk.LEFT, padx=5)
        
        btn_down = ttk.Button(control_frame, text="Bajar", command=self.move_down)
        btn_down.pack(side=tk.LEFT, padx=5)
        
        # Frame para guardar
        save_frame = ttk.Frame(main_frame)
        save_frame.pack(fill=tk.X)
        
        lbl_save = ttk.Label(save_frame, text="Archivo de salida:")
        lbl_save.pack(side=tk.LEFT, padx=(0, 5))
        
        self.output_var = tk.StringVar()
        self.entry_output = ttk.Entry(save_frame, textvariable=self.output_var, width=40)
        self.entry_output.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        btn_browse = ttk.Button(save_frame, text="Examinar", command=self.browse_output)
        btn_browse.pack(side=tk.LEFT)
        
        # Botón de combinar
        btn_merge = ttk.Button(main_frame, text="COMBINAR PDFs", command=self.merge_pdfs, style='TButton')
        btn_merge.pack(fill=tk.X, pady=(10, 0))
    
    def drop(self, event):
        # Obtener los archivos soltados
        files = self.tk.splitlist(event.data)
        for file in files:
            file_path = file.strip('{}')  # Eliminar llaves si las hay
            if file_path.lower().endswith('.pdf'):
                self.add_pdf_to_list(file_path)
            else:
                messagebox.showwarning("Archivo no válido", f"El archivo {os.path.basename(file_path)} no es un PDF.")
    
    def add_files(self):
        files = filedialog.askopenfilenames(
            title="Seleccionar archivos PDF",
            filetypes=[("Archivos PDF", "*.pdf"), ("Todos los archivos", "*.*")]
        )
        for file in files:
            self.add_pdf_to_list(file)
    
    def add_pdf_to_list(self, file_path):
        # Verificar si el archivo ya está en la lista
        if file_path in self.pdf_files:
            messagebox.showwarning(
                "Archivo duplicado",
                f"El archivo {os.path.basename(file_path)} ya está en la lista.\n\n"
                "Se agregará de todos modos, pero verifica que sea intencional."
            )
        
        # Agregar a la lista y al listbox
        self.pdf_files.append(file_path)
        self.listbox.insert(tk.END, os.path.basename(file_path))
    
    def remove_file(self):
        selected = self.listbox.curselection()
        if selected:
            index = selected[0]
            self.listbox.delete(index)
            self.pdf_files.pop(index)
    
    def move_up(self):
        selected = self.listbox.curselection()
        if selected and selected[0] > 0:
            index = selected[0]
            # Mover en la lista de archivos
            self.pdf_files[index], self.pdf_files[index-1] = self.pdf_files[index-1], self.pdf_files[index]
            # Mover en el listbox
            item = self.listbox.get(index)
            self.listbox.delete(index)
            self.listbox.insert(index-1, item)
            self.listbox.selection_set(index-1)
    
    def move_down(self):
        selected = self.listbox.curselection()
        if selected and selected[0] < len(self.pdf_files)-1:
            index = selected[0]
            # Mover en la lista de archivos
            self.pdf_files[index], self.pdf_files[index+1] = self.pdf_files[index+1], self.pdf_files[index]
            # Mover en el listbox
            item = self.listbox.get(index)
            self.listbox.delete(index)
            self.listbox.insert(index+1, item)
            self.listbox.selection_set(index+1)
    
    def browse_output(self):
        output_file = filedialog.asksaveasfilename(
            title="Guardar PDF combinado como",
            defaultextension=".pdf",
            filetypes=[("Archivos PDF", "*.pdf")]
        )
        if output_file:
            self.output_var.set(output_file)
    
    def merge_pdfs(self):
        if not self.pdf_files:
            messagebox.showerror("Error", "No hay archivos PDF para combinar.")
            return
        
        output_path = self.output_var.get().strip()
        if not output_path:
            messagebox.showerror("Error", "Debes especificar un archivo de salida.")
            return
        
        try:
            merger = PdfMerger()
            
            for pdf in self.pdf_files:
                merger.append(pdf)
            
            with open(output_path, 'wb') as f:
                merger.write(f)
            
            messagebox.showinfo("Éxito", f"PDFs combinados exitosamente en:\n{output_path}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo combinar los PDFs:\n{str(e)}")

if __name__ == "__main__":
    app = PDFMergerApp()
    app.mainloop()

Puedes hacer un programa en python que encadene uno o más archivos .PDF en un único y nuevo pdf. Que dicho programa tenga una interfaz gráfica y que los pdf a encadenar puedan ser arrastrados a un panel de dicha interfaz grafica. Que los archivos se encadene en el orden que aparecen en dicho panel de arriba hacia abajo. Y que el orden de dichos pdf se pueda mover dentro del panel, o sea que tenga funcionalidad para mover archivos hacia arriba o hacia abajo dentro del  panel. Y además si agrego un archivo que ya está en la lista a encadenar aparezca un mensaje de Warning advirtiendo esa situación, pero que no me impida adicionarlo. Además que permita también eliminar archivos de la lista de panel

# Programa para Combinar PDFs con Interfaz Gráfica

Aquí tienes un programa en Python que cumple con todos tus requisitos. Utiliza PyPDF2 para manipular los PDFs y tkinter para la interfaz gráfica con capacidad de arrastrar y soltar.

```python
import os
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from tkinter.dnd import TkinterDnD
from PyPDF2 import PdfMerger

class PDFMergerApp(TkinterDnD.Tk):
    def __init__(self):
        super().__init__()
        self.title("Combinador de PDFs")
        self.geometry("600x500")
        self.configure(bg='#f0f0f0')
        
        # Variables
        self.pdf_files = []
        
        # Estilo
        self.style = ttk.Style()
        self.style.configure('TButton', padding=5, font=('Arial', 10))
        self.style.configure('TLabel', padding=5, font=('Arial', 10))
        
        # Widgets
        self.create_widgets()
        
    def create_widgets(self):
        # Frame principal
        main_frame = ttk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Label de instrucciones
        lbl_instructions = ttk.Label(main_frame, text="Arrastra y suelta archivos PDF aquí o usa el botón 'Agregar'")
        lbl_instructions.pack(pady=(0, 10))
        
        # Listbox para mostrar los PDFs
        self.listbox = tk.Listbox(
            main_frame, 
            selectmode=tk.SINGLE, 
            height=10,
            width=60,
            bg='white',
            font=('Arial', 10),
            relief=tk.SUNKEN,
            highlightthickness=0
        )
        self.listbox.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        # Configurar DnD
        self.listbox.drop_target_register('DND_Files')
        self.listbox.dnd_bind('<<Drop>>', self.drop)
        
        # Frame de botones de control
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Botones de control
        btn_add = ttk.Button(control_frame, text="Agregar PDF(s)", command=self.add_files)
        btn_add.pack(side=tk.LEFT, padx=5)
        
        btn_remove = ttk.Button(control_frame, text="Eliminar", command=self.remove_file)
        btn_remove.pack(side=tk.LEFT, padx=5)
        
        btn_up = ttk.Button(control_frame, text="Subir", command=self.move_up)
        btn_up.pack(side=tk.LEFT, padx=5)
        
        btn_down = ttk.Button(control_frame, text="Bajar", command=self.move_down)
        btn_down.pack(side=tk.LEFT, padx=5)
        
        # Frame para guardar
        save_frame = ttk.Frame(main_frame)
        save_frame.pack(fill=tk.X)
        
        lbl_save = ttk.Label(save_frame, text="Archivo de salida:")
        lbl_save.pack(side=tk.LEFT, padx=(0, 5))
        
        self.output_var = tk.StringVar()
        self.entry_output = ttk.Entry(save_frame, textvariable=self.output_var, width=40)
        self.entry_output.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        btn_browse = ttk.Button(save_frame, text="Examinar", command=self.browse_output)
        btn_browse.pack(side=tk.LEFT)
        
        # Botón de combinar
        btn_merge = ttk.Button(main_frame, text="COMBINAR PDFs", command=self.merge_pdfs, style='TButton')
        btn_merge.pack(fill=tk.X, pady=(10, 0))
    
    def drop(self, event):
        # Obtener los archivos soltados
        files = self.tk.splitlist(event.data)
        for file in files:
            file_path = file.strip('{}')  # Eliminar llaves si las hay
            if file_path.lower().endswith('.pdf'):
                self.add_pdf_to_list(file_path)
            else:
                messagebox.showwarning("Archivo no válido", f"El archivo {os.path.basename(file_path)} no es un PDF.")
    
    def add_files(self):
        files = filedialog.askopenfilenames(
            title="Seleccionar archivos PDF",
            filetypes=[("Archivos PDF", "*.pdf"), ("Todos los archivos", "*.*")]
        )
        for file in files:
            self.add_pdf_to_list(file)
    
    def add_pdf_to_list(self, file_path):
        # Verificar si el archivo ya está en la lista
        if file_path in self.pdf_files:
            messagebox.showwarning(
                "Archivo duplicado",
                f"El archivo {os.path.basename(file_path)} ya está en la lista.\n\n"
                "Se agregará de todos modos, pero verifica que sea intencional."
            )
        
        # Agregar a la lista y al listbox
        self.pdf_files.append(file_path)
        self.listbox.insert(tk.END, os.path.basename(file_path))
    
    def remove_file(self):
        selected = self.listbox.curselection()
        if selected:
            index = selected[0]
            self.listbox.delete(index)
            self.pdf_files.pop(index)
    
    def move_up(self):
        selected = self.listbox.curselection()
        if selected and selected[0] > 0:
            index = selected[0]
            # Mover en la lista de archivos
            self.pdf_files[index], self.pdf_files[index-1] = self.pdf_files[index-1], self.pdf_files[index]
            # Mover en el listbox
            item = self.listbox.get(index)
            self.listbox.delete(index)
            self.listbox.insert(index-1, item)
            self.listbox.selection_set(index-1)
    
    def move_down(self):
        selected = self.listbox.curselection()
        if selected and selected[0] < len(self.pdf_files)-1:
            index = selected[0]
            # Mover en la lista de archivos
            self.pdf_files[index], self.pdf_files[index+1] = self.pdf_files[index+1], self.pdf_files[index]
            # Mover en el listbox
            item = self.listbox.get(index)
            self.listbox.delete(index)
            self.listbox.insert(index+1, item)
            self.listbox.selection_set(index+1)
    
    def browse_output(self):
        output_file = filedialog.asksaveasfilename(
            title="Guardar PDF combinado como",
            defaultextension=".pdf",
            filetypes=[("Archivos PDF", "*.pdf")]
        )
        if output_file:
            self.output_var.set(output_file)
    
    def merge_pdfs(self):
        if not self.pdf_files:
            messagebox.showerror("Error", "No hay archivos PDF para combinar.")
            return
        
        output_path = self.output_var.get().strip()
        if not output_path:
            messagebox.showerror("Error", "Debes especificar un archivo de salida.")
            return
        
        try:
            merger = PdfMerger()
            
            for pdf in self.pdf_files:
                merger.append(pdf)
            
            with open(output_path, 'wb') as f:
                merger.write(f)
            
            messagebox.showinfo("Éxito", f"PDFs combinados exitosamente en:\n{output_path}")
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo combinar los PDFs:\n{str(e)}")

if __name__ == "__main__":
    app = PDFMergerApp()
    app.mainloop()
```

## Requisitos para ejecutar el programa

Necesitarás instalar las siguientes bibliotecas:

```
pip install PyPDF2 tkinterdnd2
```

Nota: `tkinterdnd2` es una extensión para tkinter que permite la funcionalidad de arrastrar y soltar. En algunos sistemas operativos puede requerir instalación adicional.

## Características del programa

1. **Interfaz gráfica** con capacidad de arrastrar y soltar archivos PDF
2. **Panel de lista** que muestra los archivos en el orden de combinación
3. **Controles para**:
   - Agregar archivos (botón o arrastrar)
   - Eliminar archivos de la lista
   - Mover archivos hacia arriba o abajo en la lista
4. **Detección de archivos duplicados** con mensaje de advertencia
5. **Selección del archivo de salida** con diálogo de guardado
6. **Combinación de PDFs** usando PyPDF2

El programa es intuitivo y cumple con todos los requisitos que mencionaste.

# Alternativas para convertir tu programa Python a un ejecutable .exe para Windows 10

Existen varias herramientas para convertir scripts de Python en archivos ejecutables (.exe) para Windows. Aquí te presento las principales opciones:

## 1. PyInstaller (Recomendado)

**Ventajas:**
- Fácil de usar
- Soporte para paquetes populares
- Puede crear un solo archivo ejecutable
- Multiplataforma (Windows, Linux, macOS)

**Cómo usarlo:**
```bash
pip install pyinstaller
pyinstaller --onefile --windowed --icon=app.ico tu_programa.py
```

**Opciones útiles:**
- `--onefile`: Crea un solo archivo ejecutable
- `--windowed`: Evita que aparezca la consola (para aplicaciones GUI)
- `--icon=app.ico`: Agrega un icono personalizado
- `--add-data`: Para incluir archivos adicionales

## 2. auto-py-to-exe (Interfaz gráfica para PyInstaller)

**Ventajas:**
- Interfaz gráfica amigable
- Basado en PyInstaller
- Configuración visual

**Cómo usarlo:**
```bash
pip install auto-py-to-exe
auto-py-to-exe
```

## 3. cx_Freeze

**Ventajas:**
- Buen soporte para paquetes científicos
- Mantenido activamente

**Cómo usarlo:**
1. Crear un script setup.py:
```python
from cx_Freeze import setup, Executable

setup(
    name="PDF Merger",
    version="1.0",
    description="Combina archivos PDF",
    executables=[Executable("tu_programa.py", base="Win32GUI")]
)
```

2. Ejecutar:
```bash
pip install cx_Freeze
python setup.py build
```

## 4. Nuitka

**Ventajas:**
- Compila a código nativo (no es un empaquetador)
- Mejor rendimiento
- Binarios más pequeños

**Cómo usarlo:**
```bash
pip install nuitka
python -m nuitka --standalone --windows-disable-console --onefile tu_programa.py
```

## 5. Py2exe (Legacy)

**Ventajas:**
- Solución tradicional
- Simple para proyectos básicos

**Desventajas:**
- No se actualiza frecuentemente
- Soporte limitado para Python 3.x

**Cómo usarlo:**
1. Crear setup.py:
```python
from distutils.core import setup
import py2exe

setup(console=['tu_programa.py'])
```

2. Ejecutar:
```bash
pip install py2exe
python setup.py py2exe
```

## Recomendación específica para tu caso

Para tu aplicación de combinación de PDFs con interfaz gráfica, te recomiendo:

1. **PyInstaller** (si quieres algo rápido y sencillo)
   ```bash
   pip install pyinstaller
   pyinstaller --onefile --windowed --icon=pdf.ico tu_programa.py
   ```

2. **auto-py-to-exe** (si prefieres una interfaz gráfica para configurar)

## Consideraciones importantes

1. **Archivos adicionales**: Si tu programa usa recursos externos (como imágenes o iconos), debes incluirlos con `--add-data` en PyInstaller.

2. **Dependencias ocultas**: Algunas bibliotecas (como tkinterdnd2) pueden requerir tratamiento especial.

3. **Pruebas**: Siempre prueba el ejecutable en una máquina limpia sin Python instalado.

4. **Tamaño**: Los ejecutables pueden ser grandes porque incluyen el intérprete Python.

5. **Antivirus**: A veces los ejecutables generados pueden ser detectados falsamente como virus (especialmente con PyInstaller).